In [1]:
import pandas as pd
import numpy as np
import datetime
from itertools import cycle
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates

In [2]:
cd /home/u5/haydenp/CropBots

/home/u5/haydenp/CropBots


In [3]:
clusterID = np.load('clusterID_genotype.npy')
weather = np.load('inputs_weather_train.npy')
traits = np.load('inputs_others_train.npy')
train_yield = np.load('yield_train.npy')

## Organizing Weather Data

In [4]:
perf_records, days, variables = weather.shape

In [5]:
out_arr = np.column_stack((np.repeat(np.arange(perf_records),days),weather.reshape(perf_records*days,-1)))
out_df = pd.DataFrame(out_arr)

In [6]:
out_df = pd.DataFrame(out_arr, columns=['Performance Record','ADNI', 'AP', 'ARH', 'MDNI', 'MaxSur', 'MinSur', 'AvgSur'])

In [7]:
out_df['Performance Record'] = pd.to_numeric(out_df['Performance Record']).astype(int)

In [8]:
## Counts all of the days in Performance Record
out_df['Day'] = out_df.groupby('Performance Record').cumcount() + 1

In [9]:
out_df['Julian_date'] = out_df.groupby('Performance Record').cumcount() + 91

In [10]:
def jdtodatestd (jdate):
    fmt = '%j'
    datestd = datetime.datetime.strptime(jdate, fmt).date()
    return(datestd)

In [11]:
subset = out_df[out_df['Performance Record'] == 0]

In [12]:
date_list = []
for i, row in subset.iterrows():
    julian = int(row['Julian_date'])
    date = datetime.datetime.strptime(f'{julian}', '%j').date()
    clean_date = date.strftime("%m-%d")
    date_list.append(clean_date)

In [13]:
date_cycle = cycle(date_list)
out_df['date'] = [next(date_cycle) for cycle in range(len(out_df))]

In [14]:
out_df

,Performance Record,ADNI,AP,ARH,MDNI,MaxSur,MinSur,AvgSur,Day,Julian_date,date
0,0,169.88,0.00,50.50,814.0,57.3,32.3,43.23,1,91,04-01
1,0,12.54,0.01,84.75,93.0,51.3,36.6,43.85,2,92,04-02
2,0,321.38,0.00,65.08,986.0,56.7,28.9,42.07,3,93,04-03
3,0,153.67,0.00,63.50,777.0,67.7,43.9,52.45,4,94,04-04
4,0,6.75,0.01,71.00,118.0,63.8,38.8,47.25,5,95,04-05
...,...,...,...,...,...,...,...,...,...,...,...
19907987,93027,48.92,0.00,83.63,470.0,76.6,57.9,64.23,210,300,10-27
19907988,93027,120.21,0.00,89.42,618.0,75.8,62.6,67.39,211,301,10-28
19907989,93027,2.00,0.00,90.58,24.0,72.2,63.3,66.63,212,302,10-29
19907990,93027,258.83,0.00,82.88,884.0,80.8,63.3,68.97,213,303,10-30


## Other Trait Data

In [15]:
trait_df = pd.DataFrame(traits, columns=['Maturity Group', 'Genotype ID', 'State', 'Year', 'Location'])

In [16]:
trait_df['Year'] = pd.to_numeric(trait_df['Year'])
trait_df['Genotype ID'] = pd.to_numeric(trait_df['Genotype ID'])
trait_df['Year'] = trait_df['Year'].astype(int)
trait_df['Genotype ID'] = trait_df['Genotype ID'].astype(int)

## Yield Data

In [17]:
yield_df = pd.DataFrame(train_yield)

In [18]:
yield_df.rename(columns = {0:'Yield'}, inplace = True)

In [19]:
## Merging the two

trait_df['Yield'] = yield_df['Yield']

Adding Cluster ID to trait data

In [20]:
cluster_dict = {}
for i in range(1, 5839):
  array_index = i-1
  cID = clusterID[i-1]
  cluster_dict.update({i:cID})

In [21]:
cluster_list = []
for i in trait_df.index:
  genotype = trait_df.iloc[i]['Genotype ID']
  cluster = cluster_dict.get(genotype)
  cluster_list.append(cluster)
trait_df['Cluster'] = cluster_list

Condensed trait data frame

In [22]:
trait_df['Maturity Group'] = pd.to_numeric(trait_df['Maturity Group'])
trait_df['Maturity Group'] = trait_df['Maturity Group'].astype(int)


In [23]:
del trait_df['Location']
del trait_df['Genotype ID']

In [24]:
trait_df

,Maturity Group,State,Year,Yield,Cluster
0,3,"""KS""",2009,39.0,6.0
1,6,"""AL""",2013,54.2,6.0
2,4,"""TN""",2015,73.6,3.0
3,3,"""MO""",2014,70.9,9.0
4,3,"""NE""",2008,82.1,12.0
...,...,...,...,...,...
93023,8,"""AL""",2014,22.2,0.0
93024,4,"""IL""",2008,68.6,9.0
93025,2,"""IN""",2006,65.6,6.0
93026,5,"""LA""",2008,43.5,0.0


In [25]:
trait_df = trait_df.reset_index().rename(columns={'index':'Performance Record'})

Merging Datasets

In [45]:
daily_df = out_df
daily_df = daily_df.drop(columns=['Day', 'Julian_date',])
expanded_df = pd.concat([trait_df]*214, ignore_index=True)
expanded_df

,Performance Record,Maturity Group,State,Year,Yield,Cluster
0,0,3,"""KS""",2009,39.0,6.0
1,1,6,"""AL""",2013,54.2,6.0
2,2,4,"""TN""",2015,73.6,3.0
3,3,3,"""MO""",2014,70.9,9.0
4,4,3,"""NE""",2008,82.1,12.0
...,...,...,...,...,...,...
19907987,93023,8,"""AL""",2014,22.2,0.0
19907988,93024,4,"""IL""",2008,68.6,9.0
19907989,93025,2,"""IN""",2006,65.6,6.0
19907990,93026,5,"""LA""",2008,43.5,0.0


In [46]:
expanded_df = expanded_df.sort_values(by = ['Performance Record'])


In [47]:
expanded_df

,Performance Record,Maturity Group,State,Year,Yield,Cluster
0,0,3,"""KS""",2009,39.0,6.0
13116948,0,3,"""KS""",2009,39.0,6.0
6977100,0,3,"""KS""",2009,39.0,6.0
558168,0,3,"""KS""",2009,39.0,6.0
13023920,0,3,"""KS""",2009,39.0,6.0
...,...,...,...,...,...,...
12837863,93027,8,"""GA""",2004,40.4,8.0
12744835,93027,8,"""GA""",2004,40.4,8.0
12651807,93027,8,"""GA""",2004,40.4,8.0
12465751,93027,8,"""GA""",2004,40.4,8.0


In [48]:
expanded_df  = expanded_df.reset_index()
expanded_df

,index,Performance Record,Maturity Group,State,Year,Yield,Cluster
0,0,0,3,"""KS""",2009,39.0,6.0
1,13116948,0,3,"""KS""",2009,39.0,6.0
2,6977100,0,3,"""KS""",2009,39.0,6.0
3,558168,0,3,"""KS""",2009,39.0,6.0
4,13023920,0,3,"""KS""",2009,39.0,6.0
...,...,...,...,...,...,...,...
19907987,12837863,93027,8,"""GA""",2004,40.4,8.0
19907988,12744835,93027,8,"""GA""",2004,40.4,8.0
19907989,12651807,93027,8,"""GA""",2004,40.4,8.0
19907990,12465751,93027,8,"""GA""",2004,40.4,8.0


In [29]:
daily_df

,Performance Record,ADNI,AP,ARH,MDNI,MaxSur,MinSur,AvgSur,date
0,0,169.88,0.00,50.50,814.0,57.3,32.3,43.23,04-01
1,0,12.54,0.01,84.75,93.0,51.3,36.6,43.85,04-02
2,0,321.38,0.00,65.08,986.0,56.7,28.9,42.07,04-03
3,0,153.67,0.00,63.50,777.0,67.7,43.9,52.45,04-04
4,0,6.75,0.01,71.00,118.0,63.8,38.8,47.25,04-05
...,...,...,...,...,...,...,...,...,...
19907987,93027,48.92,0.00,83.63,470.0,76.6,57.9,64.23,10-27
19907988,93027,120.21,0.00,89.42,618.0,75.8,62.6,67.39,10-28
19907989,93027,2.00,0.00,90.58,24.0,72.2,63.3,66.63,10-29
19907990,93027,258.83,0.00,82.88,884.0,80.8,63.3,68.97,10-30


In [49]:
daily_df['Maturity Group'] = expanded_df['Maturity Group']
daily_df['State'] = expanded_df['State']
daily_df['Year'] = expanded_df['Year']
daily_df['Cluster'] = expanded_df['Cluster']
daily_df['Yield'] = expanded_df['Yield']
final_df = daily_df

In [50]:
final_df

,Performance Record,ADNI,AP,ARH,MDNI,MaxSur,MinSur,AvgSur,date,Maturity Group,State,Year,Cluster,Yield
0,0,169.88,0.00,50.50,814.0,57.3,32.3,43.23,04-01,3,"""KS""",2009,6.0,39.0
1,0,12.54,0.01,84.75,93.0,51.3,36.6,43.85,04-02,3,"""KS""",2009,6.0,39.0
2,0,321.38,0.00,65.08,986.0,56.7,28.9,42.07,04-03,3,"""KS""",2009,6.0,39.0
3,0,153.67,0.00,63.50,777.0,67.7,43.9,52.45,04-04,3,"""KS""",2009,6.0,39.0
4,0,6.75,0.01,71.00,118.0,63.8,38.8,47.25,04-05,3,"""KS""",2009,6.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19907987,93027,48.92,0.00,83.63,470.0,76.6,57.9,64.23,10-27,8,"""GA""",2004,8.0,40.4
19907988,93027,120.21,0.00,89.42,618.0,75.8,62.6,67.39,10-28,8,"""GA""",2004,8.0,40.4
19907989,93027,2.00,0.00,90.58,24.0,72.2,63.3,66.63,10-29,8,"""GA""",2004,8.0,40.4
19907990,93027,258.83,0.00,82.88,884.0,80.8,63.3,68.97,10-30,8,"""GA""",2004,8.0,40.4


In [52]:
final_df.to_csv('out.csv')